<a href="https://colab.research.google.com/github/girisiman/Deep_Learning_Tasks/blob/master/Class_Task_1_Withnoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras.datasets.mnist as mnist
import keras
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import  KerasClassifier
import os
from sklearn.metrics import accuracy_score


In [0]:
def create_model(activation = 'relu', optimizer='Adam', drop_out = 0.0, num_classes=10):
    model = Sequential()
    model.add(Dense(512, activation=activation, input_shape=(784,)))
    model.add(Dropout(drop_out))
    model.add(Dense(512, activation=activation))
    model.add(Dropout(drop_out))
    model.add(Dense(num_classes, activation=activation))
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    return model

In [10]:
def load_data(num_classes = 10):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    #y_test = keras.utils.to_categorical(y_test, num_classes)

    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = load_data()

60000 train samples
10000 test samples


In [0]:
import numpy as np

def add_noise(x_train, x_test):
  noise_factor = 0.5
  x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
  x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
  x_train_noisy = np.clip(x_train_noisy, 0., 1.)
  x_test_noisy = np.clip(x_test_noisy, 0., 1.)
  return x_train_noisy, x_test_noisy

In [0]:
x_train_noise, x_test_noise = add_noise(x_train, x_test)

In [13]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)
parameter = {'activation': ['softmax', 'elu',  'relu',],
             'optimizer': ['SGD', 'Adam', ],
             'drop_out': [ 0.5, 0.7]}
grid = GridSearchCV(estimator=model, param_grid=parameter)
grid_result = grid.fit(x_train_noise, y_train)
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_112 (Dense)            (None, 512)               401920    
_________________________________________________________________
dropout_75 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 512)               262656    
_________________________________________________________________
dropout_76 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_114 (Dense)            (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

In [14]:
best_estimator = grid.best_estimator_
best_pred_y = best_estimator.predict(x_test)
print("Accuracy: {}%".format(accuracy_score(y_test, best_pred_y)*100))


Accuracy: 79.81%
